In [3]:
#Stofzuigerzaag
#Jorn Habes & Maartje Holtslag
#22-01-2016

#Import necessary libraries
import urllib
import json
from osgeo import osr, ogr

#Create a file containing the locations and coordinates
locations = open("Locations.txt","w")

#Define a list of cities
listLocations = ["Wageningen", "Amsterdam", "Fuji", "Lima", "Bennekom", "Cairo"]

#Define a funtion to get the coordinates of a city and places them in a file
def coordinates(city, filename):
    url = urllib.urlopen("http://maps.googleapis.com/maps/api/geocode/json?address=" + city)
    data = json.loads(url.read())
    lat =  data["results"][0]["geometry"]["location"]["lat"]
    lng = data["results"][0]["geometry"]["location"]["lng"]
    string = city +' '+ str(lat) +' '+ str(lng)
    filename.write(string)
    filename.write("\n")
    return filename

#Run the function for all citites defined in the list
for place in listLocations:
    coordinates(place, locations)

#Creating the shape file using the right coordinate system
output = "Locations.shp"
driver = ogr.GetDriverByName("ESRI Shapefile")
GRS = osr.SpatialReference()
GRS.ImportFromEPSG(4326)
SHP = driver.CreateDataSource(output)
layer = SHP.CreateLayer("Locations", GRS, ogr.wkbPoint)
    
#Setting the coordinates to the layer in the shape file
locations = open("Locations.txt","r")
for line in locations:
    row = line.split()
    x, y  = [float(row[1]),float(row[2])]
    point = ogr.Geometry(ogr.wkbPoint)
    point.SetPoint(0, y, x)
    feature = ogr.Feature(layer.GetLayerDefn())
    feature.SetGeometry(point)
    layer.CreateFeature(feature)

#Closing everything    
locations.close()
SHP.Destroy()